In [1]:
!pip install -q tf-models-nightly

!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

!pip install -q tfds-nightly
!tfds --version

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.3.2 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.3.2 requires h5py<2.11.0,>=2.10.0, but you have h5py 3.1.0 which is incompatible.
tensorflow 2.3.2 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.5 which is incompatible.
tensorflow-probability 0.11.0 requires cloudpickle==1.3, but you have cloudpickle 1.6.0 which is incompatible.
TensorFlow Datasets: 4.3.0+nightly


In [2]:
import os 
import pandas as pd
import numpy as np
import cv2 as cv2
import math
import os
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.platform import flags
from tensorflow.python.platform import app
import tensorflow_datasets as tfds
import time
import re
import shutil
import tensorflow_hub as hub
from six.moves import urllib
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO
import sys
import ast
from official.vision.beta.configs import video_classification

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.6.0-dev20210514). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [3]:
train_glosses = [s.split('-')[0] for s in  os.listdir('./data/train')]
test_glosses = [s.split('-')[0] for s in  os.listdir('./data/test')]

In [4]:
print(len(train_glosses))
print(len(test_glosses))

56
25


In [5]:
labels = np.unique(np.array(train_glosses + test_glosses))

In [6]:
labels

array(['beautiful', 'hello', 'please', 'sorry'], dtype='<U9')

In [7]:
# !cd ./data/ && tfds new demo_dataset

In [8]:
!cd ./data/demo_dataset/ && tfds build --overwrite

INFO[build.py]: Loading dataset  from path: /home/jupyter/data/demo_dataset/demo_dataset.py
INFO[dataset_info.py]: Load dataset info from /home/jupyter/tensorflow_datasets/demo_dataset/1.0.0
INFO[build.py]: download_and_prepare for dataset demo_dataset/1.0.0...
INFO[native_type_compatibility.py]: Using Any for unsupported type: typing.Sequence[~T]
INFO[dataset_builder.py]: Generating dataset demo_dataset (/home/jupyter/tensorflow_datasets/demo_dataset/1.0.0)
Generating splits...:   0%|                          | 0/2 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 1 examples [00:00,  2.39 examples/s]
Generating train examples...: 2 examples [00:00,  3.69 examples/s]
Generating train examples...: 3 examples [00:00,  4.59 examples/s]
Generating train examples...: 4 examples [00:00,  5.66 examples/s]
Generating train examples...: 5 examples [00:00,  6.47 examples/s]
Generating train examples...: 6 examples [00:01,  7.23 exa

In [9]:
dataset_name = 'demo_dataset'

builder = tfds.builder(dataset_name)
num_classes = builder.info.features['label'].num_classes
num_examples = {
    name: split.num_examples
    for name, split in builder.info.splits.items()
}

In [10]:
batch_size = 8
num_frames = 8
frame_stride = 10
resolution = 224

def format_features(features):
  video = features['video']
  video = video[:, ::frame_stride]
  video = video[:, :num_frames]
  
  video = tf.reshape(video, [-1, video.shape[2], video.shape[3], 3])
  video = tf.image.resize(video, (resolution, resolution))
  video = tf.reshape(video, [-1, num_frames, resolution, resolution, 3])
  video = tf.cast(video, tf.float32) / 255.

  label = tf.one_hot(features['label'], num_classes)
  return (video, label)

train_dataset = builder.as_dataset(
    split='train',
    batch_size=batch_size,
    shuffle_files=True)
train_dataset = train_dataset.map(
    format_features,
    num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.prefetch(2)

test_dataset = builder.as_dataset(
    split='test',
    batch_size=batch_size)
test_dataset = test_dataset.map(
    format_features,
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=True)
test_dataset = test_dataset.prefetch(2)

In [36]:
sys.path.insert(1, './model')

import movinet
import movinet_model
import movinet_layers

In [12]:
os.environ['TFHUB_CACHE_DIR'] = os.getcwd()

In [ ]:
model_id = 'a2'

tf.keras.backend.clear_session()

backbone = movinet.Movinet(
    model_id=model_id,
    stochastic_depth_drop_rate=0.)
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=600,
    dropout_rate=0.)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Load pretrained weights from TF Hub
movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/base/kinetics-600/classification/1'
movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)
pretrained_weights = {w.name: w for w in movinet_hub_model.weights}
model_weights = {w.name: w for w in model.weights}
for name in pretrained_weights:
  model_weights[name].assign(pretrained_weights[name])

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=num_classes
)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Freeze all layers except for the final classifier head
for layer in model.layers[:-1]:
  layer.trainable = False
model.layers[-1].trainable = True

In [29]:
model.summary()

Model: "movinet_classifier_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
states/image (InputLayer)    [(None, None, None, None, 0         
_________________________________________________________________
movinet (Movinet)            ({'stem': (None, None, No 2738754   
_________________________________________________________________
classifier_head_1 (Classifie (None, 4)                 1320964   
Total params: 4,059,718
Trainable params: 1,320,964
Non-trainable params: 2,738,754
_________________________________________________________________


In [ ]:
num_epochs = 5

train_steps = num_examples['train'] // batch_size
total_train_steps = train_steps * num_epochs
test_steps = num_examples['test'] // batch_size

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=total_train_steps, decay_rate=0.96
)
optimizer = tf.keras.optimizers.RMSprop(
    learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

callbacks = [
    tf.keras.callbacks.TensorBoard(),
]

In [38]:
from types import MethodType

def get_config(self):
    config = {
        'backbone': self._backbone,
        'num_classes': self._num_classes,
        'input_specs': self._input_specs,
        'dropout_rate': self._dropout_rate,
        'kernel_initializer': self._kernel_initializer,
        'kernel_regularizer': self._kernel_regularizer,
        'bias_regularizer': self._bias_regularizer,
        'output_states': self._output_states,
    }
    return config

model.get_config = MethodType(get_config, model)

In [287]:
model.summary()

Model: "movinet_classifier_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
states/image (InputLayer)    [(None, None, None, None, 0         
_________________________________________________________________
movinet (Movinet)            ({'stem': (None, None, No 2738754   
_________________________________________________________________
classifier_head_1 (Classifie (None, 4)                 1320964   
Total params: 4,059,718
Trainable params: 1,320,964
Non-trainable params: 2,738,754
_________________________________________________________________


In [ ]:
results = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=num_epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks,
    validation_freq=1,
    verbose=1)

5/7 [====================>.........] - ETA: 9:12 - loss: 1.2567 - top_1: 0.6000 - top_5: 1.0000 

In [62]:
@tf.function
def serving(video_tensor):
    payload = {
      'image': video_tensor,
    }
    predictions = model(payload)
    return predictions

serving = serving.get_concrete_function(video_tensor=tf.TensorSpec([1, 5, 224, 224, 3], dtype= tf.float32, name='image'))

tf.saved_model.save(
  model,
  './model/trained_model/7',
  signatures=serving
)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadata field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadata field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: ./model/trained_model/7/assets


INFO:tensorflow:Assets written to: ./model/trained_model/7/assets


In [ ]:
imported = tf.saved_model.load('./model/trained_models/6')
f = imported.signatures["serving_default"]
arr = convert_video_to_numpy(
                filenames = ['./data/serving/video.mp4'], 
                n_frames_per_video = 5, 
                width = 224, 
                height = 224,
                n_channels = 3,
                dense_optical_flow=False)[0]

arr = np.true_divide(arr, 255)
print(arr)
# print(tf.convert_to_tensor(arr, dtype=tf.float32))
pred = f(
    image=tf.reshape(
    tensor=tf.convert_to_tensor(arr, dtype=tf.float32),
    shape=(1, 5, 224, 224, 3)))

print(pred)
layer = tf.keras.layers.Softmax()
layer(pred['output_0']).numpy()